In [1]:
import numpy as np
import networkx as nx
from scipy.optimize import linprog
from math import comb

In [2]:
q = 2
V = 9 # TODO: try V = 8 ??
delta = 0.0125 # TODO: how far we can push delta to ??

In [3]:
ALL9GRAPHS = sorted(nx.read_graph6("graph9.g6"), key = lambda G: G.number_of_edges()) # 274668 graphs on at most 9 vertices
ALLCC = np.array([len(list(nx.connected_components(G)))-1 for G in ALL9GRAPHS]) # get number of connected components of all graphs on at most 9 vertices
# isolated vertices as components

In [4]:
ALLH = ALL9GRAPHS[:19]

In [5]:
def all_colorings(V, q=2):
    """
    return all q-colorings of graphs on V vertices
    """
    for i in range(2**V):
        b = bin(i)[2:]
        yield str(0) * (V - len(b)) + b

def prume(G, C):
    """
    return pruned graph with all monochromatic edges removed and then isolated vertices removed
    """
    # remove monochromatic edge
    g = G.copy()
    for e in g.edges():
        if C[e[0]]==C[e[1]]: 
            g.remove_edge(e[0],e[1])
    
    # remove isolated vertex
    g.remove_nodes_from(list(nx.isolates(g)))
    return g

def count_iso_colorings(G, V, B):
    """
    count the number of colorings of graph G on V vertices such that the resulting subgraph
    after coloring is isomorphic to B
    """
    return np.count_nonzero(np.array([nx.is_isomorphic(B, prume(G, C)) for C in all_colorings(V, q)]))

def nodes(G):
    """
    return the number of nodes of graph G
    """
    counter=0
    for v in G.nodes():
        if G.degree(v) > 0: counter+=1
    return counter

def DC(x, q = 2, V = 9):
    """
    return the value of DC given x, q and V
    """
#     print("query",x)
#     for n in range(V+1,max(2*x+1,V+2)):
#         print("n,V,x = ", n,V,x)
    return max([comb(n,V)/(q**(n-V)*comb(n-x,V-x)) for n in range(V+1,max(2*x+1,V+2))]) 

def coef_tilde(coefs):
    """
    the list of H's that can be reduced to each H by iteratively identifying vertices from disconnected components
    """
    avail = [[0],
    [1],
    [2,3],
    [3],
    [4,5,8],
    [5,8],
    [6],
    [7,5,8],
    [8],
    [9,10,13,15],
    [10,15],
    [11],
    [12,15,17],
    [13,15],
    [14,16],
    [15],
    [16],
    [17,15],
    [18,13,15,17]]
    return [max([abs(coefs[i]) for i in avail[j]]) for j in range(len(coefs))]

In [6]:
# # generate a matrix such that rows are all graphs on at most V vertices
# # columns are sum of number of colorings of all subgraphs of G isomorphic to H
# # such that we get get P[G_q\cong H] = ISO_COUNT[G_idx][H_idx]
# ISO_COUNT = np.array([[count_iso_colorings(ALL9GRAPHS[i], V, nx.Graph(ALLH[j].edges())) for j in range(len(ALLH))] for i in range(len(ALL9GRAPHS))])/q**V
# print(ISO_COUNT.sum(axis = 1))
ISO_COUNT = np.load('k3_subg_counts_9v2.npy')[:,:19]/2**9

In [7]:
# DCs = np.array([DC(nodes(g)) for g in ALLH])
# DCs

In [8]:
# DCs = np.array([DC(nodes(g), V = V) for g in ALLH]) # TODO: why truncating DC??
DCs = np.array([0.5, 0.625, 0.7142857142857143, 0.8333333333333334, 0.8333333333333334, 1.0, 0.7142857142857143, 0.8333333333333334, 0.2976190476190476, 1.0, 0.2976190476190476, 0.8333333333333334, 1.0, 0.2976190476190476, 0.8333333333333334, 0.5, 0.5, 0.5, 1.0])
DCs

array([0.5       , 0.625     , 0.71428571, 0.83333333, 0.83333333,
       1.        , 0.71428571, 0.83333333, 0.29761905, 1.        ,
       0.29761905, 0.83333333, 1.        , 0.29761905, 0.83333333,
       0.5       , 0.5       , 0.5       , 1.        ])

In [9]:
def check_condition_2(coefs):
    """
    check that all graphs with at most 3 edges have mu less than or equal to 1
    """
    sums = np.apply_along_axis(lambda x: np.sum(x*coefs), 1, ISO_COUNT[:9])
    return (np.max(np.abs(sums)[1:]) <=1) and sums[0] == 7
def check_condition_3(coefs):
    """
    check that all graphs with more than tC2 edges and at most V vertices have mu at most 1-delta
    check that the maximum bound is at most 1-delta by iterating over all graphs on at most V certices
    """
    small_sums = np.apply_along_axis(lambda x: np.sum(x*coefs), 1, ISO_COUNT[9:]) # graphs > tC2 edges and <= V vertices
    large_sums = np.apply_along_axis(lambda x: np.sum(x*coef_tilde(coefs)*DCs), 1, ISO_COUNT[1:]) # check graphs > tC2 edges and > V vertices
    print("bound on small",np.max(np.abs(small_sums)),"bound on big",np.max(large_sums/q**(ALLCC_old[1:])),"threshold", 1-delta)
    return (np.max(large_sums/q**(ALLCC_old[1:])) <= 1 - delta) and (np.max(np.abs(small_sums)) <= 1 - delta)

In [10]:
coefs = np.array([7.0, -5.0, -1.0, 1.7, 3.0, 0.0, 0.0, 0.3, 0.0, -0.2, 0.0, 0.0, 0.0, 0.0, -3.7, 0.0, 0.0, 0.0, -0.75])
# print(coef_tilde(coefs))
res=check_condition_2(coefs)
res2 = check_condition_3(coefs)
print(res, res2)

NameError: name 'ALLCC_old' is not defined

In [ ]:
all_avail_coeff = []
# all disconnected H have 0 coefficients
for x3 in np.arange(-1.0,2.0,0.4):
    for x4 in np.arange(-2,5,1):
        for x7 in np.arange(-1.0,1.0,0.2):
            for x9 in np.arange(-1.0,1.0,0.4):
                for x12 in np.arange(-1,2,1):
                    for x14 in np.arange(-4.0,2.0,0.5):
                        for x18 in np.arange(-1.0,1.0,0.5):
                            coefs = np.array([7.0,-5,-1,x3,x4,0.0,0.0,x7,0.0,x9,0.0,0.0,x12,0.0,x14,0.0,0.0,0.0,x18])
                            res=check_condition_2(coefs)
                            res2 = check_condition_3(coefs)
                            if res and res2:
                                all_avail_coeff.append(coefs)
with open('coeff_res.npy', 'wb') as f:
    np.save(f,np.array(all_avail_coeff))